# 1. self-defined function

In [51]:
import time
import datetime
import pandas as pd
from influxdb import InfluxDBClient

#UTC时间格式
UTC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"
UTC_FORMAT_NO_S = "%Y-%m-%dT%H:%M:%SZ"

#将正八区时间转换为标准格林威治时间
def shift_UTC_time(stime):
    tTZ = +8
    if not isinstance(stime, datetime.datetime):
        if '.' in stime:
            stime = datetime.datetime.strptime(stime, UTC_FORMAT)
        else:
            stime = datetime.datetime.strptime(stime, UTC_FORMAT_NO_S)
    return  stime - datetime.timedelta(hours=tTZ)

#将标准格林威治时间转换为正八区时间
def shift_local_time(stime):
    tTZ = +8
    if not isinstance(stime, datetime.datetime):
        if '.' in stime:
            stime = datetime.datetime.strptime(stime, UTC_FORMAT)
        else:
            stime = datetime.datetime.strptime(stime, UTC_FORMAT_NO_S)
    return  stime + datetime.timedelta(hours=tTZ)

#从数据库读数据
def smartdb_quary(Start_quary_time,
                  End_quary_time, 
                  DeviceId_quary,
                  host,
                  port,
                  password, 
                  user =,
                  dbname = 'GeoTest.GeoRetention',
                  Print_Flag = 0):
    
    """Instantiate a connection to the InfluxDB."""
    # SELECT "raw_ADC" AS "mean_raw_ADC" FROM "GeoTest"."GeoRetention"."SecondlyReading" WHERE time >'2019-03-27T16:00:00.000Z' AND time <'2019-03-27T16:10:00.000Z' AND "DeviceId"='1000'
    
    #Start_quary_time = shift_UTC_time(Start_quary_time) - datetime.timedelta(seconds=0.5)
    #End_quary_time = shift_UTC_time(End_quary_time) + datetime.timedelta(seconds=0.5)
    
    Start_quary_time = shift_UTC_time(Start_quary_time)
    End_quary_time = shift_UTC_time(End_quary_time)
    query = 'SELECT \"raw_ADC\" AS \"mean_raw_ADC\" FROM \"GeoTest\".\"GeoRetention\".\"SecondlyReading\" WHERE ' + \
            'time >' + ' \'' + Start_quary_time.strftime(UTC_FORMAT) + '\'' + ' AND ' + \
            'time <' + ' \'' + End_quary_time.strftime(UTC_FORMAT) + '\'' +  ' AND ' + \
            '\"DeviceId\"=' + '\'' + DeviceId_quary + '\''

    # Debug Print Only
    # print("Host ip is: " + host)
    # print("Port is: " + str(port))

    client = InfluxDBClient(host, port, user, password, dbname)

    # Debug Print Only
    if Print_Flag == 1:
        print("Querying data: " + query)
    Influxdb_result = client.query(query)
    
    Influxdb_points = list(Influxdb_result.get_points(measurement = 'SecondlyReading'))

    del Influxdb_result
    Result_df = pd.DataFrame(Influxdb_points)
    #转换为正八区时间
    Result_df["time"] = Result_df["time"].apply(shift_local_time)
    Result_df["time"] = Result_df["time"].apply(lambda x: x.strftime(UTC_FORMAT))
    del Influxdb_points
    return Result_df

SyntaxError: invalid syntax (<ipython-input-51-947a20cbc434>, line 37)

In [56]:
import os
import csv
import shutil

def data_load(dir_to_sava_data = r'.\data3-29_222',
              experiment_record = r'experience3_29.csv',
              DAY_TIME = '2019-03-29T',
              deviceId_list = ['1000','1001','1002','1003'],
              label_list = ['CloseOutDoor', 'OpenOutDoor',
                            'OpenToiletDoor', 'CloseToiletDoor',
                            'OpenKitchenDoor', 'CloseKitchenDoor', 
                            'KitchenLight',
                            'FlushToilet', 'WashHand', 'Bathe', 
                            'BoilWater', 'MicrowaveOven', 'InductionCooker',
                            'NoAppliance'],
              host='', 
              port=0,
              password = '',
              user = ''):
    
    with open(experiment_record, newline='') as csvfile:       
        #创建一个储存所有数据的文件夹,如果存在，就先删除该文件夹及其中文件
        if os.path.exists(dir_to_sava_data):
            shutil.rmtree(dir_to_sava_data)
        os.mkdir(dir_to_sava_data)
        
        reader = list(csv.reader(csvfile))
        Time_list = []
        for index in range(len(reader)):
        #for index in range(8):
            if index <2 or index%2==1:
                continue
            #read data & remove ''
            first_list = list(filter(None, reader[index]))
            second_list = list(filter(None, reader[index+1]))
            label = first_list[0]
            deviceId = first_list[1]

            #只下载指定deviceId、label的数据
            if label not in label_list:  continue
            if deviceId not in deviceId_list:  continue
                
            HourMin = first_list[2:]
            HourMin = [DAY_TIME+x for x in HourMin]
            Seconds = second_list[2:]
            Time = [(datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M") + datetime.timedelta(seconds=float(y))) for x,y in zip(HourMin,Seconds)]
            Time_list.append(Time)

            path = dir_to_sava_data + '\\'+ label
            if not os.path.exists(path):
                os.mkdir(path)
            for i in range(len(Time)//2):
                Start_quary_time = Time[2*i]
                End_quary_time = Time[2*i+1]
                df = smartdb_quary(Start_quary_time, End_quary_time, deviceId, host, port, password, user)
                df.to_csv(path + r'\%ssample%d.csv' % (deviceId,i+1))
            print(label + ' ' + deviceId + ':  completed' )

# 2. mian program

In [ ]:
data_load(dir_to_sava_data = r'.\data3-29_222',
          experiment_record = r'experience3_29.csv',
          DAY_TIME = '2019-03-29T',
          deviceId_list = ['1000','1001','1002','1003'],
          label_list = ['CloseOutDoor', 'OpenOutDoor',
                        'OpenToiletDoor', 'CloseToiletDoor',
                        'OpenKitchenDoor', 'CloseKitchenDoor', 
                        'KitchenLight',
                        'FlushToilet', 'WashHand', 'Bathe', 
                        'BoilWater', 'MicrowaveOven', 'InductionCooker',
                        'NoAppliance'],
          host='', 
          port=0,
          password = '',
          user = '')